In [5]:
import torch
import random

random_seed = 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(random_seed)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(random_seed)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(random_seed)
 
# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

Using device: cuda


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from torch.utils.data import DataLoader
from rich.progress import track

from binary_dataset import BinaryDataset

batch_size = 32
tokens_per_paragraph = 256  # Fixed number of tokens per paragraph

train_dataset = BinaryDataset("pan21/train", "pan21/train")
val_dataset = BinaryDataset("pan21/validation", "pan21/validation")

# dataloader seems wonky, always returns tuple len 2 with default collate fn
def collate_fn(batch):
    # Separate each tuple into two lists
    x = [item[0] for item in batch]
    y = torch.stack([item[1] for item in batch]).view(len(batch), 1)

    return x, y
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

Output()

Using author file: pan21\train_authors
Using embedding dir: pan21\train_embeddings


Output()

Using author file: pan21\validation_authors
Using embedding dir: pan21\validation_embeddings


In [8]:
import torch
import torch.nn as nn
import cupy as cp
from cupy.fft import fft

class ParagraphLabelingModelWithBERT(nn.Module):
    def __init__(self, tokens_per_paragraph, output_dim, fourier=False):
        super(ParagraphLabelingModelWithBERT, self).__init__()
        
        self.fourier = fourier
        self.tokens_per_paragraph = tokens_per_paragraph

        bert_embedding_dimension = 768
        input_size = bert_embedding_dimension*self.tokens_per_paragraph*2
        self.fc = nn.Linear(input_size, output_dim, device=DEVICE)
        # self.fc = nn.Linear(input_size, output_dim*512, device=DEVICE)
        # self.fc2 = nn.Linear(output_dim*512, output_dim*256, device=DEVICE)
        # self.fc3 = nn.Linear(output_dim*256, output_dim, device=DEVICE)

    def forward(self, paragraph_pair):
        # Tokenize paragraphs
        # print(paragraphs)

        # batches come as lists, individual examples as tuples
        if isinstance(paragraph_pair, tuple) and len(paragraph_pair) == 2:
            paragraph_pair = [paragraph_pair]

        # print(f"{len(paragraph_pair)=} {len(paragraph_pair[0])=}")
        # print(f"{paragraph_pair[0]=}")
        
        if self.fourier:
            paragraph_pair = [
                    (torch.tensor(cp.real(fft(cp.asarray(p1), axis=1)), device=DEVICE), 
                    torch.tensor(cp.real(fft(cp.asarray(p2), axis=1)), device=DEVICE))
                for (p1, p2) in paragraph_pair
                ]

        x = self.fc(torch.stack([torch.flatten(torch.cat((p[0].squeeze(0), p[1].squeeze(0)), dim=0)) for p in paragraph_pair]))
        # print(f"{x.shape=}")

        
        
        # x = self.fc3(self.fc2(self.fc(x)))
        # x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [9]:
import datetime
from rich.progress import Progress
from pathlib import Path

# Model parameters
bert_model_name = 'bert-base-uncased'
output_dim = 1    # Number of classes
lr_exps = range(4, 8)
# lr_exps = [6]
# fourier = True
num_epochs = 10

for fourier in (False, True):
    for lr_exponent in lr_exps:
        lr = 1/10**lr_exponent
        model = ParagraphLabelingModelWithBERT(tokens_per_paragraph, output_dim, fourier=fourier)

        criterion = nn.BCELoss()
        # learning rate is important, .001 led to always outputting 1
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        with Progress() as pb:
            total_task = pb.add_task('Total', total=num_epochs)

            for epoch in range(num_epochs):
                epoch_task = pb.add_task(f'Epoch {epoch}', total=len(dataloader))

                model.train()
                for inputs, targets in dataloader:
                    # print(f"{type(inputs)=} {len(inputs)=}")
                    # print(f"{type(inputs[0])=}")
                    # print(f"{targets.shape=}")
                    
                    optimizer.zero_grad()

                    # Forward pass
                    outputs = model(inputs)
                    # print(f"{outputs.shape=}")

                    # Compute the loss
                    loss = criterion(outputs, targets)

                    # Backward pass and optimization
                    loss.backward()
                    optimizer.step()

                    pb.update(epoch_task, advance=1)

                metadata = {
                    'epoch': epoch,
                    'loss': loss.item(),
                    'optimizer_state': 'Adam',
                    'lr': lr,
                    'fourier': fourier,
                    'tokens_per_paragraph': tokens_per_paragraph,
                }
                state_dict = model.state_dict()
                state_dict['metadata'] = metadata

                print(f"{model.state_dict()=}")

                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
                pb.update(total_task, advance=1)
                torch.save(model.state_dict(), Path(f"{datetime.datetime.now().strftime('%Y_%m_%d-%I_%M_%S_%p')}_e{epoch}.torch"))


Output()

Epoch [1/10], Loss: 6.3133

Epoch [2/10], Loss: 3.6810

Epoch [3/10], Loss: 1.3389

Epoch [4/10], Loss: 1.2435

...

Epoch [6/10], Loss: 0.8979

Epoch [7/10], Loss: 0.8536

Epoch [8/10], Loss: 0.8155

Epoch [9/10], Loss: 0.7824

Epoch [10/10], Loss: 0.7534

In [20]:
eval_dict = torch.load("2024_07_28-02_56_01_AM_e9.torch")
output_dim = 1
model = ParagraphLabelingModelWithBERT(tokens_per_paragraph, output_dim)
model.load_state_dict(eval_dict)
model.eval()

val_loss = 0
criterion = nn.BCELoss()

for input, target in val_dataloader:
    with torch.no_grad():
        val_outputs = model(input)
        
        val_loss += criterion(val_outputs.view(-1, output_dim), target).item()

print(f'Validation Loss: {val_loss/len(val_dataloader):.4f}')

Validation Loss: 0.7144


In [21]:
from prediction import build_prediction_dict
prediction_dict = build_prediction_dict(model, val_dataset)

Output()

In [22]:
from prediction import map_predictions_to_json
dict_of_jsons_result = {f"problem-{problem_num}":map_predictions_to_json(prediction_map) for problem_num, prediction_map in prediction_dict.items()}

from evaluation.evaluator import read_ground_truth_files
truth = read_ground_truth_files("pan21/validation")


from evaluation.evaluator import compute_score_single_predictions, compute_score_multiple_predictions
task1_result = compute_score_single_predictions(truth, dict_of_jsons_result, 'multi-author')
task2_result = compute_score_multiple_predictions(truth, dict_of_jsons_result, 'changes', labels=[0, 1])
task3_result = compute_score_multiple_predictions(truth, dict_of_jsons_result, 'paragraph-authors', labels=[1, 2, 3, 4])

print(
    # f'Model: {model_path.stem}\n' +
    f'\tTask 1 Score: {task1_result}\n'+
    f'\tTask 2 Score: {task2_result}\n'+
    f'\tTask 3 Score: {task3_result}\n'
)


	Task 1 Score: 0.5338889899909008
	Task 2 Score: 0.4517440956597779
	Task 3 Score: 0.2946667018857537

